In [1]:
import argparse
from logging import getLogger
from tqdm import tqdm
import torch
import numpy as np
import pandas as pd
import pickle

from recbole.quick_start import load_data_and_model
from recbole.utils import init_logger, get_model, get_trainer, init_seed, set_color
from recbole.data import create_dataset
from recbole.utils.case_study import full_sort_topk, full_sort_scores

import logging
from logging import getLogger

import torch
import pickle

from recbole.config import Config
from recbole.data import create_dataset, data_preparation, save_split_dataloaders, load_split_dataloaders
from recbole.utils import init_logger, get_model, get_trainer, init_seed, set_color

import copy
from recbole.data.interaction import Interaction, cat_interactions

from collections import OrderedDict
import os

pd.set_option('display.max_rows', None)

In [41]:
sasrec_log = 'output/lanzhou/0121-exp/up-/down-/SASRec.meta-test.log'
sasrecfeat_log = 'output/lanzhou/0121-exp/up-/down-/SASRecFeat.meta-test.log'
fusion_log = 'output/lanzhou/0121-exp/up-/down-/fusion-9-1.meta-test.log'

In [3]:
def get_result(log):
    with os.popen(f'tail -n 1 {repr(log)}') as f:
        content = f.read()
    return eval(content.split('test result: ')[1])

In [42]:
sasrec_result = get_result(sasrec_log)
sasrecfeat_result = get_result(sasrecfeat_log)
fusion_result = get_result(fusion_log)

In [27]:
tasks = list(sasrec_result.keys())

In [29]:
sasrec_answer = [
    int(sasrec_result[task]['recall@10'])
    for task in tasks
]

In [30]:
sasrecfeat_answer = [
    int(sasrecfeat_result[task]['recall@10'])
    for task in tasks
]

In [43]:
fusion_answer = [
    int(fusion_result[task]['recall@10'])
    for task in tasks
]

In [44]:
df = pd.DataFrame({
    'tasks': tasks,
    'SASRec': sasrec_answer,
    'SASRecFeat': sasrecfeat_answer,
    'fusion': fusion_answer,
})
df.head()

,tasks,SASRec,SASRecFeat,fusion
0,268685745,0,0,0
1,2366968655,1,0,1
2,88707795,0,0,0
3,290734363,0,0,0
4,630528524,0,0,0


In [46]:
df[(df['SASRec'] == 1) | (df['SASRecFeat'] == 1) | (df['fusion'] == 1)]

,tasks,SASRec,SASRecFeat,fusion
1,2366968655,1,0,1
5,551437851,1,1,1
9,386772516,1,0,1
11,166465577,1,1,1
19,2867495880,1,1,1
26,2490084176,1,1,1
37,256641652,1,0,1
40,2985333025,1,1,1
50,780308103,1,1,1
55,927701980,1,0,1


In [33]:
sasrec_mask = (df['SASRec'] == 1)
sasrecfeat_mask = (df['SASRecFeat'] == 1)

In [34]:
sasrec_mask.sum()

32

In [35]:
sasrecfeat_mask.sum()

17

In [36]:
(sasrecfeat_mask & (~sasrec_mask)).sum()

4

In [37]:
(sasrec_mask & (~sasrecfeat_mask)).sum()

19

In [38]:
(sasrec_mask & sasrecfeat_mask).sum()

13

In [39]:
len(df)

189

0.1693121693121693